## Imports

In [ ]:
!pip install openpyxl
!pip install PyArabic
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git &> /dev/null
!pip install emoji 
!pip install Arabic-Stopwords
!pip install tkseem
!pip install tnkeeh

In [ ]:
!pip3 install fr-word-segment

In [ ]:
!pip install pyspellchecker

In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from keras.preprocessing.text import Tokenizer

import nltk
import string
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from fastai.text.all import *

import sklearn
import regex as re
from unicodedata import normalize

import torch
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch.optim.lr_scheduler import CyclicLR
from torchvision import models
from sklearn.metrics import accuracy_score, confusion_matrix
import os
import gensim


# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import GRU,MaxPooling1D,GlobalMaxPooling1D,Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras import callbacks
from keras.utils.vis_utils import plot_model

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer 
from sklearn.metrics import roc_auc_score, accuracy_score,roc_curve, auc, plot_confusion_matrix, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from sklearn.manifold import TSNE
from sklearn.naive_bayes import MultinomialNB
from keras.preprocessing.text import Tokenizer
import emoji
from keras.models import Model
import seaborn as sn
import pyarabic.araby as ar
import tkseem as tk
import tnkeeh as tn
from nltk.stem.isri import ISRIStemmer
from spellchecker import SpellChecker
from wordsegment import load,segment
from keras.layers.merge import Concatenate
import tensorflow as tf
load()

## Loading Data

In [ ]:
print(tf. __version__)

In [ ]:
df_ar = pd.read_csv('/kaggle/input/twitter/ar_dataset.csv')
df_ar2 = pd.read_excel('/kaggle/input/twitter/arr.xlsx')

#main data
df_fr = pd.read_csv('/kaggle/input/twitter/fr_dataset.csv')
df_fr2 = pd.read_csv('/kaggle/input/twitter/french_tweets.csv')

## Exploring data

#### French Dataset

###### 1) Main dataset

In [ ]:
df_fr.head()

In [ ]:
df_fr.describe()

In [ ]:
print('Size of the dataset:')
len(df_fr)

In [ ]:
#test if the data contains null values
print('Nan value',df_fr.isnull().sum())

In [ ]:
#take a look at the column of the dataframe to see the features
df_fr.columns

In [ ]:
df_fr['sentiment'].value_counts()

In [ ]:
#### Class distribution 
cum = df_fr['target'].value_counts().to_frame()
cum['HITId'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='HITId',y='target',ax=ax)

###### 2) Assest dataset

In [ ]:
df_fr2.head()

In [ ]:
print('Size of the dataset:')
len(df_fr2)

In [ ]:
#test if the data contains null values
print('Nan value',df_fr2.isnull().sum())

###### Note : 
in this dataset we don't have as much features as in the previous one!

#### Arabic dataset

###### 1) Main dataset

In [ ]:
df_ar.head()

In [ ]:
df_ar.describe()

In [ ]:
print('Size of the dataset:')
len(df_ar)

In [ ]:
#test if the data contains null values
print('Nan value',df_ar.isnull().sum())

In [ ]:
df_ar['sentiment'].value_counts()

In [ ]:
#### Class distribution 
cum = df_ar['target'].value_counts().to_frame()
cum['HITId'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='HITId',y='target',ax=ax)

###### 2) Assest dataset

In [ ]:
df_ar2.head()

In [ ]:
print('Size of the dataset:')
len(df_ar2)

In [ ]:
#test if the data contains null values
print('Nan value',df_ar2.isnull().sum())

## Data processing

Since our task is a binary classification: negative or positive tweet, we will create a dataset that contains both negative and positive tweet. To do this, we will merge the previous dataset with the assest one that contains positive tweets. 

While merging, the assest doesn't contain the columns : HTDID, sentiment, directness, annotator_sentiment and group,some NaN values appeared. To avoid them these values will be replaced in each column with an appripriate value, for instance:


sentiment-> 'normal', 

directness->'direct',

groupe-> 'nothing' and 

annotator_sentiment->'indifference'

#### French Dataset

In [ ]:
# negative tweets contained
# we put target = 0 negative
neg = df_fr.loc[df_fr['target']!='normal','target'] = 0
neg = df_fr
neg.head()

In [ ]:
#positive tweets contained in the assest dataset
pos = df_fr2.loc[df_fr2['label']==1]
#since we have a several amount of data, we take a part of it
n = len(pos)
pos= pos[0:int(n/100)]
len(pos)
pos= pos.rename(columns={'label':'target','text':'tweet'})

In [ ]:
# merging the two dataframes
data_fr = pd.concat([neg,pos], ignore_index=True, sort=False)
#print(len(data_fr))
data_fr.head()

In [ ]:
# fixing nan values
data_fr.loc[data_fr['sentiment'].isnull().values==True,'sentiment'] = 'normal'
data_fr.loc[data_fr['directness'].isnull().values==True,'directness'] = 'direct'
data_fr.loc[data_fr['group'].isnull().values==True,'group'] = 'nothing'
data_fr.loc[data_fr['annotator_sentiment'].isnull().values==True,'annotator_sentiment'] = 'indifference'
data_fr = sklearn.utils.shuffle(data_fr)

In [ ]:
#data_fr = data_fr.drop(columns='HITId')
data_fr.reindex().head()

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(data_fr["target"].value_counts(),labels=data_fr["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig("distribution des données dans les différentes classes.png")
plt.show()

In [ ]:
def oversample(df):
    classes = df.target.value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df['target'] == key]) 
    classes_sample = []
    for i in range(1,len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
    final_df = final_df.reset_index(drop=True)
    return final_df

In [ ]:
data_fr = oversample(data_fr)
data_fr = data_fr.drop(columns={'HITId'})
data_fr.head()

In [ ]:
# distribution of classes: 0,1
cum = data_fr['target'].value_counts().to_frame()
cum['tweet'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='tweet',y='target',ax=ax)

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(data_fr["target"].value_counts(),labels=data_fr["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig("distribution des données dans les différentes classes.png")
plt.show()

#### Arabic dataset

In [ ]:
neg = df_ar.loc[df_ar['target']!='normal','target'] = 0
neg = df_ar
neg.head()

In [ ]:
df_ar2.head()

In [ ]:
#positive tweets contained in the assest dataset
df_ar2.loc[df_ar2['Sentiment']=='Positive']
df_ar2.loc[df_ar2['Sentiment']=='Positive','Sentiment'] = 1
pos = df_ar2.loc[df_ar2["Sentiment"]==1]
pos= pos.rename(columns={'Sentiment':'target','Feed':'tweet'})
pos = pos.drop(columns={'ID'})

In [ ]:
pos.head()

In [ ]:
# merging the two dataframes
data_ar = pd.concat([neg,pos], ignore_index=True, sort=False)
#print(len(data_ar))
data_ar.head()

In [ ]:
data_ar.loc[data_ar['sentiment'].isnull().values==True,'sentiment'] = 'normal'
data_ar.loc[data_ar['directness'].isnull().values==True,'directness'] = 'direct'
data_ar.loc[data_ar['group'].isnull().values==True,'group'] = 'nothing'
data_ar.loc[data_ar['annotator_sentiment'].isnull().values==True,'annotator_sentiment'] = 'indifference'
data_ar = sklearn.utils.shuffle(data_ar)
data_ar.drop(columns='HITId')

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(data_ar["target"].value_counts(),labels=data_ar["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig(" ara distribution des données dans les différentes classes.png")
plt.show()

In [ ]:
data_ar = oversample(data_ar)
data_ar = data_ar.drop(columns={'HITId'})
data_ar.head()

In [ ]:
# distribution of classes: 0,1
cum = data_ar['target'].value_counts().to_frame()
cum['tweet'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='tweet',y='target',ax=ax)

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(data_ar["target"].value_counts(),labels=data_ar["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig(" ara distribution des données dans les différentes classes.png")
plt.show()

In [ ]:
data_fr.columns

In [ ]:
#encoding labels
le = preprocessing.LabelEncoder()
data_fr.sentiment = le.fit_transform(data_fr.sentiment)
data_fr.directness = le.fit_transform(data_fr.directness)
data_fr.annotator_sentiment = le.fit_transform(data_fr.annotator_sentiment)
data_fr.group = le.fit_transform(data_fr.group)
data_fr

In [ ]:
data_ar.sentiment = le.fit_transform(data_ar.sentiment)
data_ar.directness = le.fit_transform(data_ar.directness)
data_ar.annotator_sentiment = le.fit_transform(data_ar.annotator_sentiment)
data_ar.group = le.fit_transform(data_ar.group)
data_ar.head()

In [ ]:
def hash_fix(h):
    h1 = re.sub(r'[0-9]+', '', h)
    h2 = re.sub(r'#', '', h1)
    h3 = segment(str(h2))
    h4 = ' '.join(map(str, h3)) 
    return h4

In [ ]:
tok_ar = tk.WordTokenizer()
tok_ar.train('/kaggle/input/twitter/ar_dataset.csv')
def prepro_ar(tweet):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    
    user = "*@user" # l'etoile se met au début, car l'arabe se lit de droite vers la gauche
    tweet = re.sub(arabic_diacritics, '', str(tweet))
    tweet = re.sub('r'+user, "", tweet)
    tweet = re.sub(r'(.)\1+', "", tweet) 
    tweet = ar.strip_tashkeel(tweet)
    tweet = ar.strip_tatweel(tweet)
    tweet = tweet.replace("@", " ")
    tweet = tweet.replace("_", " ")
    tweet = re.sub("ى", "ي", tweet)
    tweet = re.sub("ؤ", "ء", tweet)
    tweet = re.sub("ئ", "ء", tweet)
    tweet = re.sub("ة", "ه", tweet)
    tweet = re.sub("گ", "ك", tweet)
    tweet = tweet.replace("آ", "ا")
    tweet = tweet.replace("إ", "ا")
    tweet = tweet.replace("أ", "ا")
    tweet = tweet.replace("ؤ", "و")
    tweet = tweet.replace("ئ", "ي")
    tweet = nltk.tokenize.word_tokenize(tweet)
    tweet = [ISRIStemmer().suf32(w) for w in tweet]

    return tweet

In [ ]:
contractions = {
        'administration':'admin',
        'avec':'ac',
        'beaucoup':'bp',
        'c’est-à-dire':'cad',
        'cependant':'cpd',
        'chose':'ch',
        'conclusion':'ccl',
        'confer ':'cf',
        'court terme':'ct',
        'dans':'ds',
        'dedans':'dd',
        'définition':'déf',
        'et cetera':'etc',
        'être':'ê',
        'exemple':'ex',
        'extérieur':'ext',
        'font':'ft',
        'général':'gal',
        'gouvernement':'gouv',
        'grand':'gd',
        'groupe':'gp',
        'identique':'idel',
        'introduction':'intro',
        'jour':'jr',
        'long terme':'lt',
        'lorsque':'lsq',
        'mais':'ms',
        'même':'^m',
        'moyen terme':'mt',
        'nombre':'nb',
        'nombreux':'nbx',
        'nombre':'nb',
        'nombreux':'nbx',
        'observation':'obs',
        'ordre du jour':'oj',
        'page':'p',
        'parce que':'pcq',
        'pendant':'pdt',
        'personne':'pers',
        'point':'pt',
        'peut-être':'pê',
        'pour':'pr',
        'pourtant':'prtt',
        'quand':'qd',
        'quantité':'qté',
        'que':'q',
        'quelqu’un':'qqn',
        'quelque chose':'qqch',
        'quelque':'qq',
        'quelquefois':'qqf',
        'question':'quest',
        'rendez-vous':'rdv',
        'responsabilité':'respité',
        'seulement':'slt',
        'solution':'sol',
        'sont':'st',
        'sous':'ss',
        'souvent':'svt',
        'temps':'tps',
        'toujours':'tjrs',
        'tous':'ts',
        'tout':'tt',
        'toute':'tte',
        'toutes':'ttes',
        'vous':'vs',
        'le':'l\'',
        'me':'m\'',
        'de':'d\'',
        'te':'t\'',
        'se':'s\'',
        'ce':'c\'',
        'ne':'n\'',
        'que':'qu\'',
        'jusque':'jusqu\'',
        'lorsque':'lorsqu\'',
        'puisque':'puisqu\'',
        'quelque':'quelqu\'',
        'quoique':'quoiqu\'',
        'parce que':'parce qu\'',
        'tel que':'tel qu\'',
        'telle que':'telle qu\'',       
        'faculte':'fac',
        'bien':'bin',    

}

In [ ]:
french_stopwords = nltk.corpus.stopwords.words('french')
lemmatizer = FrenchLefffLemmatizer()
spell = SpellChecker(language='fr')

In [ ]:
def prepro_fr(tweet):
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # normalize unicode characters
    tweet = normalize('NFD', tweet).encode('ascii','ignore')
    tweet = tweet.decode('UTF-8')
    #demojize
    tweet = emoji.demojize(tweet)
    if "#" in tweet:
        tweet = hash_fix(tweet)
    tweet = tweet.replace('user', '')
    tweet = tweet.replace('@user', '')
    tweet = tweet.replace('url', '')
    tweet = re.sub( r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'',tweet)
    # convert to lower case
    tweet = tweet.lower()
    tweet = tweet.replace('\'', '\' ') 
    # remove punctuation
    tweet = re.sub(r"\p{P}", lambda m: "-" if m.group(0) == "-" else "", tweet)
    # tokenization
    tweet = nltk.tokenize.word_tokenize(tweet)
    #contractions
    tweet = [list(contractions.keys())[list(contractions.values()).index(word)] if word in contractions.values() else word for word in tweet]
    # stop words
    tweet = [w for w in tweet if w not in french_stopwords]
    # remove non-printable chars form each token
    tweet = [re_print.sub('', w) for w in tweet]
    # lemmatization
    tweet = [lemmatizer.lemmatize(w) for w in tweet]
    return tweet

In [ ]:
data_ar.tweet = data_ar.tweet.apply(lambda t: prepro_ar(t))

In [ ]:
data_fr.tweet = data_fr.tweet.apply(lambda t: prepro_fr(t))

In [ ]:
tweets = data_fr.tweet.values

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz

In [ ]:
!gunzip cc.ar.300.vec.gz


In [ ]:
!wc -l cc.ar.300.vec

In [ ]:
f_ar = open('cc.ar.300.vec', encoding='utf-8')

In [ ]:
data_fr

In [ ]:
data_ar.columns

In [ ]:
labels_ar = data_ar.target.values
data_ar = data_ar.drop(columns={'target'})

In [ ]:
X_train_ar,X_test_ar, y_train_ar,y_test_ar = train_test_split(data_ar,labels_ar,test_size=0.1)
# text ---> integer sequence
X_train_ar1 = X_train_ar.tweet.values
X_test_ar1 = X_test_ar.tweet.values

for i in range(len(X_train_ar1)):
    X_train_ar1[i] = tok_ar.encode_sentences(X_train_ar1[i]).flatten().astype(np.float32)
for i in range(len(X_test_ar1)):
    X_test_ar1[i]  = tok_ar.encode_sentences(X_test_ar1[i]).flatten().astype(np.float32)

In [ ]:
X_train_ar2 = X_train_ar[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
X_test_ar2 = X_test_ar[['sentiment', 'directness', 'annotator_sentiment', 'group']].values

In [ ]:
tweets_length_ar= [len(X_train_ar1[i]) for i in range(len(X_train_ar1))]
max_seq_len_ar = max(tweets_length_ar)
print(max_seq_len_ar)

In [ ]:
#preparing embedding  model!
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz

In [ ]:
!gunzip cc.fr.300.vec.gz

In [ ]:
!wc -l cc.fr.300.vec

In [ ]:
f_fr = open('cc.fr.300.vec', encoding='utf-8')

In [ ]:
#loading pretrained model to word vecs
embeddings_index_fr = {}
for line in tqdm(f_fr):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    embeddings_index_fr[word]=vector
f_fr.close()

print('found %s word vectors' % len(embeddings_index_fr))

In [ ]:
data_fr.head()

In [ ]:
labels_fr = data_fr.target.values
data_fr = data_fr.drop(columns={'target'})
X_train_fr,X_test_fr, y_train_fr,y_test_fr = train_test_split(data_fr,labels_fr,test_size=0.1)

In [ ]:
X_train_fr1 = X_train_fr.tweet.values
X_test_fr1 = X_test_fr.tweet.values

tok_fr = Tokenizer()
tok_fr.fit_on_texts(data_fr.tweet.values)

for i in range(len(X_train_fr1)):
    X_train_fr1[i] = tok_fr.texts_to_sequences(X_train_fr1[i])
    X_train_fr1[i] = pad_sequences(X_train_fr1[i], maxlen=300)
for i in range(len(X_test_fr1)):
    X_test_fr1[i] = tok_fr.texts_to_sequences(X_test_fr1[i])
    X_test_fr1[i] = pad_sequences(X_test_fr1[i], maxlen=300)
    

In [ ]:
X_train_fr2 = X_train_fr[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
X_test_fr2 = X_test_fr[['sentiment', 'directness', 'annotator_sentiment', 'group']].values

In [ ]:
#counting the length of the tweet and taking the max
tweets_length_fr= [len(X_train_fr1[i]) for i in range(len(X_train_fr1))]
max_seq_len_fr = max(tweets_length_fr)
print(max_seq_len_fr)

In [ ]:
embed_dim = 300 

In [ ]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found_fr = []
word_index_fr = tok_fr.word_index
nb_words_fr = len(word_index_fr)+1
embedding_matrix_fr = np.zeros((nb_words_fr, embed_dim))
for word, i in word_index_fr.items():
    if i >= nb_words_fr:
        continue
    embedding_vector = embeddings_index_fr.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_fr[i] = embedding_vector
    else:
        words_not_found_fr.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix_fr, axis=1) == 0))
print(len(embedding_matrix_fr))

In [ ]:
input_fr_1 = Input(shape=(max_seq_len_fr,))
input_ar_1 = Input(shape=(max_seq_len_ar,))
input_2 = Input(shape=(4,))

In [ ]:
embedding_layer_fr = Embedding(
                            nb_words_fr, 
                            embed_dim,  
                            weights=[embedding_matrix_fr],
                            input_length=max_seq_len_fr,
                            trainable=False
                    )(input_fr_1)

In [ ]:
def NN(lang,input_1,input_2):
    model = Sequential()
    if lang=='AR':  
        embedding_layer = Embedding(input_dim = tok_ar.vocab_size,output_dim = embed_dim, input_length=max_seq_len_ar)(input_ar_1)
        #model.add(embedding_layer)
    else:
        embedding_layer = embedding_layer_fr
        #model.add(embedding_layer_fr)
    # 1st bi-LSTM layer
    bi_lstm = Bidirectional(LSTM(128,return_sequences=True))(embedding_layer)
    bilstm_dropout = Dropout(0.5)(bi_lstm)
    x = GlobalMaxPool1D()(bilstm_dropout)
    #FC layers
    dense_layer_1 = Dense(10, activation='relu')(input_2)
    #model.add(dense_layer_1)
    dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)
    #model.add(Dense(64, activation='relu')(dense_layer_1))
    #model.add(Dropout(0.5))
    
    concat_layer = Concatenate()([x, dense_layer_2])
    dense_layer_3 = Dense(10, activation='relu')(concat_layer)
    output = Dense(1, activation='sigmoid')(dense_layer_3)
    #model.add(Dense(1, activation='sigmoid'))
    model = Model(inputs=[input_1, input_2], outputs=output)
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
    return model

In [ ]:
#tweets_ar = np.asarray([X_train_ar[0][0].astype('float32').flatten() for i in range(len(X_train_ar))])
#tweets_fr = np.asarray([X_train_fr[0][0].astype('float32').flatten() for i in range(len(X_train_fr))])

In [ ]:
early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
num_epochs = 10
batch_size = 128

model_fr = NN('FR',input_fr_1,input_2)
model_ar = NN('AR',input_ar_1,input_2)

In [ ]:
len(X_train_ar1[0])

In [ ]:
print('Arabic:')
Xar = [np.asarray(X_train_ar1), np.asarray(X_train_ar2)]
history_ar = model_ar.fit(Xar,
                          y_train_ar.astype(np.float32),
                          batch_size=batch_size,
                          epochs=num_epochs,
                          validation_split=0.15,
                          callbacks = [early],
                          verbose=1)

In [ ]:
print('French:')
Xfr = [X_train_fr1,X_train_fr2]
history_fr = model_fr.fit(Xfr,
                          y_train_fr.astype(np.float32),
                          batch_size=batch_size,
                          epochs=num_epochs,
                          validation_split=0.15,
                          callbacks = [early],
                          verbose=1)

In [ ]:
def plot_acc_loss(history):
    """
    Plot accuracy and loss of a model
    @params:
            - history: history of the model
    @return:
            plots
    """
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    l = list(history.history.keys())
    print(l)
    # accuracy plot
    ax[0].plot(history.history[l[1]])
    ax[0].plot(history.history[l[3]])
    ax[0].set_title('model accuracy')
    ax[0].set_ylabel('accuracy')
    ax[0].set_xlabel('epoch')
    ax[0].legend(['train', 'test'], loc='upper left')
    # loss plot
    ax[1].plot(history.history[l[0]])
    ax[1].plot(history.history[l[2]])
    ax[1].set_title('model loss')
    ax[1].set_ylabel('loss')
    ax[1].set_xlabel('epoch')
    ax[1].legend(['train', 'test'], loc='upper left')

In [ ]:
plot_acc_loss(history_ar)

In [ ]:
plot_acc_loss(history_fr)

In [ ]:
def predicted_label(model,x):
    pred =np.array(model.predict(x))
    lab_pred = []
    for i in pred:
        if i>=0.5:
            lab_pred.append(1)
        else:
            lab_pred.append(0)
    return lab_pred

In [ ]:
def plot_cm(model,x,y):
    pred = model.predict(x)
    y_pred = predicted_label(model,x)
    cm = confusion_matrix(list(y),y_pred)  
    sn.heatmap(cm, annot=True)

In [ ]:
print("Confusion Matrix for french dataset")
plot_cm(model_fr,X_test_seq_fr,y_test_fr)

In [ ]:
print("Confusion Matrix for arabic dataset")
plot_cm(model_ar,X_test_seq_ar,y_test_ar)

In [ ]:
def classify_fr(tweet):
    cleaned =  prepro_fr(tweet)
    tok_fr.fit_on_texts(cleaned )
    com_seq = tok_fr.texts_to_sequences([cleaned])
    com_pad = pad_sequences(com_seq,maxlen=max_seq_len_fr, padding='post')
    pred = model_fr.predict(com_pad[0])
    return pred

In [ ]:
classify_fr("gros batard")

In [ ]:
def classify_ar(tweet):
    cleaned =  prepro_ar(tweet)
    com_seq= tok_ar.encode_sentences(cleaned)
    com_pad = pad_sequences(com_seq,maxlen=max_seq_len_ar, padding='post')
    pred = model_ar.predict(com_pad)[0][0]
    print(pred)

In [ ]:
#classify_ar("اانه جميل جدا")
cleaned =  prepro_ar("اانه جميل جدا")
com_seq= tok_ar.encode_sentences(cleaned, out_length = max_seq_len_ar)
com_pad = pad_sequences(com_seq,maxlen=max_seq_len_ar, padding='post')
pred = model_ar.predict(com_pad[0])[0]
com_pad 

In [ ]:
classify_ar("شعبك اغبياء")